In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
df = pd.read_csv("../Datasets/flavornet_dataset.csv")
df.dropna(inplace=True) # Drop N/A values

# Clean up the odor names by removing whitespace and commas

odors_series = df['Odor']
odors_series = odors_series.str.split(",")

for index in odors_series.index:
    # Clean up the odor names for each molecule
    odors_series[index] = [odor.strip().strip(',').lower() for odor in odors_series[index]]

In [3]:
all_odors = [] # List of all the odors present in the dataset
def get_labels_counts(array):
    for label in array:
        if label not in all_odors:
            all_odors.append(label)

odors_series.apply(get_labels_counts)

mlb = MultiLabelBinarizer() # Encoding the labels as boolean arrays
encoding = mlb.fit_transform(odors_series.tolist())
classes = mlb.classes_

In [4]:

# Need to add it to the data frame because we shuffle before splitting
encoding_df = pd.DataFrame(encoding, columns=classes)
df.reset_index(drop=True, inplace=True)
encoding_df.reset_index(drop=True, inplace=True)
final_df = pd.concat([df, encoding_df], axis=1) # Adding the matrix as columns to the data frame

# final_df is now the data frame to apply algorithms on

In [5]:
labels_to_drop = all_odors.copy()
labels_to_drop += ['Odorant', 'Odor', 'SMILES representation'] # List of labels to drop for x_train and x_test

x = final_df.drop(labels=labels_to_drop, axis=1)
y = final_df.loc[:, all_odors]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [6]:
# Apply neural network model #

model = Sequential()
model.add(Dense(50, activation='relu', input_dim=x_train.shape[1]))
model.add(Dense(y_train.shape[1], activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

model.fit(x_train, y_train,epochs=2,batch_size=400)
preds = model.predict(x_test) # Predictions
preds[preds>=0.5] = 1
preds[preds<0.5] = 0

Epoch 1/2
2/2 [==============================] - 0s 2ms/step - loss: 5.5041 - accuracy: 0.0118
Epoch 2/2
2/2 [==============================] - 0s 6ms/step - loss: 3.7682 - accuracy: 0.0094
